# **Ejercicios de pair programming Módulo 3 Sprint 1**
## **Regreción Lineal IV: Lección 4**

En este ejercicio de pair programming seguiréis usando el dataset que seleccionasteis.



El objetivo de este pairprgramming es que evaluéis si vuestro set de datos cumple todas las asunciones que se deben cumplir para hacer una regresión lineal. 

Recordamos que estas asunciones son:

- Normalidad (ya la deberíais haber evaluado)
- Homogeneidad de varianzas
- Independencia de las variables
  
Cada asunción la deberéis testar de forma visual y analítica.